### Load Cleaned Dataset

In [325]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

test = pd.read_csv("../data/cleaned/test.csv")
control = pd.read_csv("../data/cleaned/control.csv")
demo = pd.read_csv("../data/cleaned/demo.csv")
exp = pd.read_csv("../data/cleaned/exp.csv")

### Client Profile
1. Who are the primary clients using this online process?
2. Are the primary clients younger or older, new or long-standing?
3. Next, carry out a client behaviour analysis to answer any additional relevant questions you think are important.


In [ ]:
demo_exp = pd.merge(demo, exp, on="client_id")

In [ ]:
demo_exp.client_id.nunique()

In [ ]:
sns.histplot(demo['clnt_age'], kde=True, bins=30, color='red').set_title("Client's Age Distribution")

In [ ]:
sns.histplot(data=demo_exp, x="clnt_age", hue="Variation", kde=True, bins=30, color='red').set_title("Client's Age Distribution (Test vs Control)");

In [ ]:
sns.histplot(demo_exp['clnt_age'][demo_exp["Variation"]=="Control"], kde=True, bins=30, color='red').set_title("Client's Age Distribution (Control Group)")

In [ ]:
sns.histplot(demo_exp['clnt_age'][demo_exp["Variation"]=="Test"], kde=True, bins=30, color='red').set_title("Client's Age Distribution (Test Group)")

Age distribution between Control & Test Group are relatively the same with the whole clients data in demo.

In [ ]:
demo_exp.describe().round()

In [ ]:
sns.histplot(demo['clnt_tenure_yr'], kde=True, bins=30, color='red')

In [ ]:
sns.scatterplot(demo, x='clnt_age', y='clnt_tenure_yr')

In [ ]:
pd.crosstab(demo["calls_6_mnth"], demo["logons_6_mnth"])

In [ ]:
sns.scatterplot(demo, x='clnt_age', y='bal')

In [ ]:
sns.scatterplot(demo[demo["bal"]<2000000], x='clnt_age', y='bal')

### Performace Metrics

1. Completion Rate: The proportion of users who reach the final ‘confirm’ step.
2. Time Spent on Each Step: The average duration users spend on each step.
3. Error Rates: If there’s a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

### 1. Completion Rate: The proportion of users who reach the final ‘confirm’ step.

Assumption:
- No matter from which visitor_id or visit_id, as long as the client_id follow the process_step sequentially (start > step 1 > step 2 > step 3 > confirm), the process is considered complete.

In [326]:
# Create pivot dataframe (if the client_id has duplicate in 'visit_id' and 'process_step' -> keep the last)
test_pivot = test.drop_duplicates(subset=['client_id', 'visitor_id', 'visit_id','process_step'], keep='last')
test_pivot = test_pivot.pivot(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='date_time')
test_pivot = test_pivot.reset_index()
test_pivot = test_pivot[['client_id','visitor_id','visit_id','start','step_1','step_2','step_3','confirm']]

#Create dummy variable for each step (1=there is timestamp in particular step; 0=otherwise)
test_pivot['start_count'] = test_pivot[['start']].notna().sum(axis=1)
test_pivot['step_1_count'] = test_pivot[['step_1']].notna().sum(axis=1)
test_pivot['step_2_count'] = test_pivot[['step_2']].notna().sum(axis=1)
test_pivot['step_3_count'] = test_pivot[['step_3']].notna().sum(axis=1)
test_pivot['confirm_count'] = test_pivot[['confirm']].notna().sum(axis=1)

# Sum of count of log by each client_id (Note: some client_id may have >1 count in particular step)
test_count_log = test_pivot.groupby('client_id')[['start_count','step_1_count','step_2_count','step_3_count','confirm_count']].sum().reset_index()

# Create dummy variable which identify client_id who reached specific step
CR_test = test_count_log[['client_id', 'start_count', 'step_1_count', 'step_2_count','step_3_count', 'confirm_count']]
CR_test['start'] = CR_test['start_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_test['step_1'] = CR_test['step_1_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_test['step_2'] = CR_test['step_2_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_test['step_3'] = CR_test['step_3_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_test['confirm'] = CR_test['confirm_count'].apply(lambda x: 1 if x >= 1 else 0)

# Not counting client_id who does not have timestamp on the previous step (sequentially)
CR_test.loc[CR_test['step_3'] == 0, 'confirm'] = 0
CR_test.loc[CR_test['step_2'] == 0, 'step_3'] = 0
CR_test.loc[CR_test['step_1'] == 0, 'step_2'] = 0
CR_test.loc[CR_test['start'] == 0, 'step_1'] = 0

# Final Completion Rate
Completion_Rate_test = CR_test[['start','step_1','step_2','step_3','confirm']].sum().reset_index()
Completion_Rate_test.columns = ['process_step', 'num_users']
Completion_Rate_test[["CR_test"]] = round(Completion_Rate_test[["num_users"]] / Completion_Rate_test["num_users"][0] *100,2)

In [327]:
# Create pivot dataframe (if the client_id has duplicate in 'visit_id' and 'process_step' -> keep the last)
control_pivot = control.drop_duplicates(subset=['client_id', 'visitor_id', 'visit_id','process_step'], keep='last')
control_pivot = control_pivot.pivot(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='date_time')
control_pivot = control_pivot.reset_index()
control_pivot = control_pivot[['client_id','visitor_id','visit_id','start','step_1','step_2','step_3','confirm']]

#Create dummy variable for each step (1=there is timestamp in particular step; 0=otherwise)
control_pivot['start_count'] = control_pivot[['start']].notna().sum(axis=1)
control_pivot['step_1_count'] = control_pivot[['step_1']].notna().sum(axis=1)
control_pivot['step_2_count'] = control_pivot[['step_2']].notna().sum(axis=1)
control_pivot['step_3_count'] = control_pivot[['step_3']].notna().sum(axis=1)
control_pivot['confirm_count'] = control_pivot[['confirm']].notna().sum(axis=1)

# Sum of count of log by each client_id (Note: some client_id may have >1 count in particular step)
control_count_log = control_pivot.groupby('client_id')[['start_count','step_1_count','step_2_count','step_3_count','confirm_count']].sum().reset_index()

# Create dummy variable which identify client_id who reached specific step
CR_control = control_count_log[['client_id', 'start_count', 'step_1_count', 'step_2_count','step_3_count', 'confirm_count']]
CR_control['start'] = CR_control['start_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_control['step_1'] = CR_control['step_1_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_control['step_2'] = CR_control['step_2_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_control['step_3'] = CR_control['step_3_count'].apply(lambda x: 1 if x >= 1 else 0)
CR_control['confirm'] = CR_control['confirm_count'].apply(lambda x: 1 if x >= 1 else 0)

# Not counting client_id who does not have timestamp on the previous step (sequentially)
CR_control.loc[CR_control['step_3'] == 0, 'confirm'] = 0
CR_control.loc[CR_control['step_2'] == 0, 'step_3'] = 0
CR_control.loc[CR_control['step_1'] == 0, 'step_2'] = 0
CR_control.loc[CR_control['start'] == 0, 'step_1'] = 0

# Final Completion Rate
Completion_Rate_control = CR_control[['start','step_1','step_2','step_3','confirm']].sum().reset_index()
Completion_Rate_control.columns = ['process_step', 'num_users']
Completion_Rate_control[["CR_control"]] = round(Completion_Rate_control[["num_users"]] / Completion_Rate_control["num_users"][0] *100,2)

In [328]:
Completion_Rate_test

,process_step,num_users,CR_test
0,start,26679,100.00
1,step_1,24201,90.71
2,step_2,22243,83.37
3,step_3,20834,78.09
4,confirm,18445,69.14


In [329]:
Completion_Rate_control

,process_step,num_users,CR_control
0,start,23397,100.00
1,step_1,20082,85.83
2,step_2,18633,79.64
3,step_3,17356,74.18
4,confirm,15329,65.52


### 2. Time Spent on Each Step: The average duration users spend on each step.

Assumption:

In [335]:
# Create pivot dataframe (if the client_id has duplicate in 'visit_id' and 'process_step' -> keep the last)
test_pivot = test.drop_duplicates(subset=['client_id', 'visitor_id', 'visit_id','process_step'], keep='last')
test_pivot = test_pivot.pivot(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='date_time')
test_pivot = test_pivot.reset_index()
test_time = test_pivot[['client_id','visitor_id','visit_id','start','step_1','step_2','step_3','confirm']]

steps = ['start','step_1','step_2','step_3','confirm']
for step in steps:
    test_time[step] = pd.to_datetime(test_time[step])
    
test_time["start_dur"] = test_time["step_1"] - test_time["start"]
test_time["step_1_dur"] = test_time["step_2"] - test_time["step_1"] 
test_time["step_2_dur"] = test_time["step_3"] - test_time["step_2"] 
test_time["step_3_dur"] = test_time["confirm"] - test_time["step_3"] 


durs = ['start_dur','step_1_dur','step_2_dur','step_3_dur']
for dur in durs:
    test_time[dur] = test_time[dur].apply(lambda x: x if x >= pd.Timedelta(0) else pd.NaT)
    #test_time[dur] = test_time[dur].apply(lambda x: f"{int(x.total_seconds() // 60):02}:{int(x.total_seconds() % 60):02}" if pd.notnull(x) else np.nan)
    
time_test = test_time[['start_dur','step_1_dur','step_2_dur','step_3_dur']].mean().reset_index()
time_test.columns = ['process_step', 'duration']

In [349]:
# Create pivot dataframe (if the client_id has duplicate in 'visit_id' and 'process_step' -> keep the last)
control_pivot = control.drop_duplicates(subset=['client_id', 'visitor_id', 'visit_id','process_step'], keep='last')
control_pivot = control_pivot.pivot(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='date_time')
control_pivot = control_pivot.reset_index()
control_time = control_pivot[['client_id','visitor_id','visit_id','start','step_1','step_2','step_3','confirm']]

steps = ['start','step_1','step_2','step_3','confirm']
for step in steps:
    control_time[step] = pd.to_datetime(control_time[step])
    
control_time["start_dur"] = control_time["step_1"] - control_time["start"]
control_time["step_1_dur"] = control_time["step_2"] - control_time["step_1"] 
control_time["step_2_dur"] = control_time["step_3"] - control_time["step_2"] 
control_time["step_3_dur"] = control_time["confirm"] - control_time["step_3"] 


durs = ['start_dur','step_1_dur','step_2_dur','step_3_dur']
for dur in durs:
    control_time[dur] = control_time[dur].apply(lambda x: x if x >= pd.Timedelta(0) else pd.NaT)
    #control_time[dur] = control_time[dur].apply(lambda x: f"{int(x.total_seconds() // 60):02}:{int(x.total_seconds() % 60):02}" if pd.notnull(x) else np.nan)
    
time_control = control_time[['start_dur','step_1_dur','step_2_dur','step_3_dur']].mean().reset_index()
time_control.columns = ['process_step', 'duration']

In [348]:
time_test

,process_step,duration
0,start_dur,0 days 00:00:49.330020809
1,step_1_dur,0 days 00:00:50.851985402
2,step_2_dur,0 days 00:01:42.118102065
3,step_3_dur,0 days 00:02:23.443006308


In [351]:
time_control

,process_step,duration
0,start_dur,0 days 00:00:54.578368944
1,step_1_dur,0 days 00:00:51.818625393
2,step_2_dur,0 days 00:01:39.330432829
3,step_3_dur,0 days 00:02:15.819548872


### 3. Error Rate